## 데이터 집계와 그룹 연산
- 하나 이상의 키(함수, 배열, DataFrame 컬럼 이름)을 이용해서 pandas 객체를 여러 조각으로 나누는 방법
- 합계, 평균, 표준편차 사용자 정의 함수 같은 그룹 요약 통계를 계산하는 방법
- 정규화, 선형회귀 , 등급 또는 부분집합 선택 같은 집단 내 변형이나 다른 조작을 적용하는 방법
- 변위치 분석과 다른 통계 집단 분석을 수행하는 방법

- **기본 Pandas 함수 에서 제거**
- **시계열 데이터의 집계의 경우 groupby = 리샘플링방법(resampling)**
- **사용방법    ==>     데이터.groupby(['열이름']).적용함수()**

![그룹연산예시](imgs/그룹연산예시.png)

## 데이터 선언

In [3]:
import numpy as np
import pandas as pd

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

df = pd.DataFrame({'key1':['a','a','b','b','a'],
                   'key2':['one','two','one','two','one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.657446,-0.233373
1,a,two,-0.918170,2.207287
2,b,one,-1.452056,0.928736
3,b,two,0.088773,2.234303
4,a,one,-0.459922,0.120033


## groupby 객체

In [4]:
# grouped 변수는 GroupBy객체이고, 그룹연산을 위해 필요한 정보를 가지고 있는 것.
grouped = df['data1'].groupby(df['key1'])
grouped

# 그룹별 평균 구하기.
grouped.mean()

key1
a   -0.678513
b   -0.681641
Name: data1, dtype: float64

## groupby 옵션

In [5]:
# mean
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

# size
sizes = df['data1'].groupby([df['key1'], df['key2']]).size()
sizes

key1  key2
a     one    -0.558684
      two    -0.918170
b     one    -1.452056
      two     0.088773
Name: data1, dtype: float64

key1  key2
a     one     2
      two     1
b     one     1
      two     1
Name: data1, dtype: int64

## Unstack 계층해제하기

In [6]:
# unstack으로 계층해제하기
means.unstack()
print(type(means.unstack())) # DataFrame or Series 형태 반환

key2,one,two
key1,,
a,-0.558684,-0.918170
b,-1.452056,0.088773


<class 'pandas.core.frame.DataFrame'>


## Groupby 그룹 순회하기

In [7]:
# 그룹간 순회하기
for name, group in df.groupby('key1'):
    print('name',name)
    print(group,'\n')
    
    
# 그룹순회 - dict형으로 가져오기
pieces = dict(list(df.groupby('key1')))
pieces

name a
  key1 key2     data1     data2
0    a  one -0.657446 -0.233373
1    a  two -0.918170  2.207287
4    a  one -0.459922  0.120033 

name b
  key1 key2     data1     data2
2    b  one -1.452056  0.928736
3    b  two  0.088773  2.234303 



{'a':   key1 key2     data1     data2
 0    a  one -0.657446 -0.233373
 1    a  two -0.918170  2.207287
 4    a  one -0.459922  0.120033,
 'b':   key1 key2     data1     data2
 2    b  one -1.452056  0.928736
 3    b  two  0.088773  2.234303}

## 컬럼이나 컬럼의 일부만 선택하기

In [26]:
# 여러 컬럼 선택해서 (key1, key2) data2 groupby
df.groupby(['key1','key2'])[['data2']].mean()

# 컬럼의 일부만 선택해서 groupby
df['data1'].groupby(df['key1']).mean()

data2
key1 key2          
a    one  -0.812568
     two   0.739611
b    one  -1.095206
     two   1.385332

key1
a    0.384919
b   -0.982966
Name: data1, dtype: float64

## 사전과 Series에서 그룹핑하기
- 그룹 정보는 배열이 아닌 형태로 존재하기도 한다.
- dict 형태로 존재할 때 키값으로 데이터 모으고 value로 컬럼 데이터

In [9]:
people = pd.DataFrame(np.random.randn(5,5), 
                      columns=['a','b','c','d','e'],
                     index = ['Joe','Steve','Wes','Jim','Travis'])

# 2행 (1번,2번 컬럼) nan
people.iloc[2:3, [1,2]] = np.nan

# dict
mapping = {'a':'red','b':'red','c':'blue','d':'blue',
           'e':'red','e':'red','f':'orange'}
mapping
print('groupby 기준 dictionary')

# people
people
print('people data')

# groupyby - dict
by_column = people.groupby(mapping, axis=1)
by_column.sum()
print('groupby 계산결과')

{'a': 'red', 'b': 'red', 'c': 'blue', 'd': 'blue', 'e': 'red', 'f': 'orange'}

groupby 기준 dictionary


,a,b,c,d,e
Joe,-1.407376,1.903233,-0.224021,1.166461,2.346123
Steve,-2.173440,1.466605,-0.485765,-1.712693,-0.785088
Wes,0.217075,NaN,NaN,-1.349136,0.222517
Jim,-1.213006,-0.233767,-1.155126,-0.238673,-0.508133
Travis,-1.275066,0.948932,0.908479,0.730325,-1.100186


people data


,blue,red
Joe,0.942440,2.841980
Steve,-2.198458,-1.491922
Wes,-1.349136,0.439592
Jim,-1.393799,-1.954906
Travis,1.638803,-1.426320


groupby 계산결과


## 함수로 그룹핑하기
- 색인값으로 row 인덱스를 쓴다는 것 명심

In [22]:
# 이름의 길이별로 합계 통계
people.groupby(len).sum() # len 이라고 쓰면 그 기준이 알아서 index의 len이 됨.

# 이름의 길이별로 합계 통계 2
ls = [3,5,3,3,6]
people.groupby(ls).sum()

# len 과 key_list 분류로 이어짐
key_list=['one','one','one','two','two']
people.groupby([len, key_list]).min()

,a,b,c,d,e
3,-2.403306,1.669466,-1.379147,-0.421348,2.060506
5,-2.173440,1.466605,-0.485765,-1.712693,-0.785088
6,-1.275066,0.948932,0.908479,0.730325,-1.100186


,a,b,c,d,e
3,-2.403306,1.669466,-1.379147,-0.421348,2.060506
5,-2.173440,1.466605,-0.485765,-1.712693,-0.785088
6,-1.275066,0.948932,0.908479,0.730325,-1.100186


a         b         c         d         e
3 one -1.407376  1.903233 -0.224021 -1.349136  0.222517
  two -1.213006 -0.233767 -1.155126 -0.238673 -0.508133
5 one -2.173440  1.466605 -0.485765 -1.712693 -0.785088
6 two -1.275066  0.948932  0.908479  0.730325 -1.100186

## 데이터 집계
- 데이터 집계는 배열로부터 스칼라값을 만들어내는 모든 데이터 변환 작업을 말함.
- 집계함수는 일반적으로 좀더 느리게 동작하는데, 중간 데이터를 생성하는 과정에서 함수 호출이나 데이터 정렬 같은 오버헤드가발생하기 때문
- count, sum, mean, median, std, var, min, max, prod(NA가 아닌 값들의 곱), first(NA가 아닌 값들 중 첫번째값), last(NA가 아닌 값들 중 마지막값)
- quantile은 groupby를 위해서 구현되진않았고, Series메서드로 사용가능.

In [29]:
df
print('원래 데이터')

grouped = df.groupby('key1')

grouped['data1'].quantile(0.9)

,key1,key2,data1,data2
0,a,one,-0.657446,-0.233373
1,a,two,-0.918170,2.207287
2,b,one,-1.452056,0.928736
3,b,two,0.088773,2.234303
4,a,one,-0.459922,0.120033


원래 데이터


key1
a   -0.499427
b   -0.065310
Name: data1, dtype: float64

## 자신만의 데이터 집계함수 사용 
- 배열의 aggregate나 agg 메서드에 해당 함수 넘기기

In [33]:
# 최대에서 최소빼기
def max_sub_min(arr):
    return arr.max() - arr.min()

# agg 함수 - 사용자정의함수
grouped.agg(max_sub_min)

# aggregate 함수 - mean
grouped.aggregate('mean')

,data1,data2
key1,,
a,0.458247,2.440660
b,1.540829,1.305567


,data1,data2
key1,,
a,-0.678513,0.697982
b,-0.681641,1.581519


## map함수, apply 함수, applymap 함수

### 1. map함수 
- DataFrame이 아닌 Series 타입에서만 사용 가능.
    - Sereis란 = 인덱스(index) + 값(value)

### 2. apply함수
- apply함수는 Series 객체에서도 사용가능하고, DataFrame 객체에서도 사용가능하다는 
- **apply 함수는 DataFrame의 행/열 기반으로 작동 / applymap은 DataFrame에서 요소별로 작동**
- **가장 일반적인 GroupBy 메서드의 목적. apply 하기 위해서**

 
### 3. applymap 함수
- DataFrame에서 요소별로 작동함

### 요약
- map은 Series형에서의 각각의 데이터에 대한 접근.
- apply가 DataFrame의 행, 열에 대한 함수 적용 결과
- applymap DataFrame에 접근하되 각각의 데이터에 접근


In [54]:
### map 함수는 Series 형에대해서만 적용 가능.

frame = pd.DataFrame(np.random.randn(4, 3), 
                  columns=list('bde'), 
                  index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

f_map = lambda x: '%.2f' % x

# series라 작동가능
frame['e'].map(f_map)


,b,d,e
Utah,-0.729602,-0.457167,1.811120
Ohio,-0.407900,-0.231570,0.833774
Texas,-0.779708,-0.923775,-0.319383
Oregon,-0.129744,-1.580398,0.990451


Utah       1.81
Ohio       0.83
Texas     -0.32
Oregon     0.99
Name: e, dtype: object

In [55]:
### apply 함수
frame

# f 함수 정의
f_apply = lambda x: x.max() - x.min()

# 세로 열에 대해 f함수
frame.apply(f_apply, axis=0) 

# 가로 행에 대해 f함수
frame.apply(f_apply, axis=1) 


,b,d,e
Utah,-0.729602,-0.457167,1.811120
Ohio,-0.407900,-0.231570,0.833774
Texas,-0.779708,-0.923775,-0.319383
Oregon,-0.129744,-1.580398,0.990451


b    0.649965
d    1.348828
e    2.130502
dtype: float64

Utah      2.540721
Ohio      1.241674
Texas     0.604393
Oregon    2.570849
dtype: float64

In [60]:
### applymap 함수
# apply가 DataFrame의 행, 열로 접근, applymap 은 그 각각의 데이터에 접근
frame

f_applymap = lambda x: '%.3f' % x
frame.applymap(f_applymap)

,b,d,e
Utah,-0.729602,-0.457167,1.811120
Ohio,-0.407900,-0.231570,0.833774
Texas,-0.779708,-0.923775,-0.319383
Oregon,-0.129744,-1.580398,0.990451


,b,d,e
Utah,-0.730,-0.457,1.811
Ohio,-0.408,-0.232,0.834
Texas,-0.780,-0.924,-0.319
Oregon,-0.130,-1.580,0.990


## 변위치 분석과 버킷 분석
- cut함수와 qcut 메서드 이용. 선택한 크기만큼 표본 변위치에 따라 데이터를 나눔
- N등분 및, group apply, unstack

In [69]:
frame2 = pd.DataFrame({'data1':np.random.randn(100),
                       'data2':np.random.randn(100)})

# 4등분 하기
quartiles = pd.cut(frame2.data1, 4)
quartiles

0     (-1.639, -0.547]
1      (-0.547, 0.546]
2      (-0.547, 0.546]
3       (0.546, 1.638]
4     (-1.639, -0.547]
            ...       
95      (0.546, 1.638]
96    (-2.736, -1.639]
97     (-0.547, 0.546]
98    (-1.639, -0.547]
99     (-0.547, 0.546]
Name: data1, Length: 100, dtype: category
Categories (4, interval[float64]): [(-2.736, -1.639] < (-1.639, -0.547] < (-0.547, 0.546] < (0.546, 1.638]]

In [75]:
# 4등분 groupby 결과
grouped = frame2.data2.groupby(quartiles)
grouped.mean()

# min, max, count, mean 구해서 리턴
def get_stats(group):
    return {'min':group.min(), 'max':group.max(),
             'count':group.count(), 'mean':group.mean()}

grouped_all = grouped.apply(get_stats)
grouped_all

# unstack 으로 group형 풀기
grouped_all.unstack()

data1
(-2.736, -1.639]    0.151850
(-1.639, -0.547]   -0.090475
(-0.547, 0.546]    -0.032136
(0.546, 1.638]     -0.025809
Name: data2, dtype: float64

data1                  
(-2.736, -1.639]  min       0.048876
                  max       0.254824
                  count     2.000000
                  mean      0.151850
(-1.639, -0.547]  min      -1.617784
                  max       2.612791
                  count    23.000000
                  mean     -0.090475
(-0.547, 0.546]   min      -1.684224
                  max       1.574596
                  count    49.000000
                  mean     -0.032136
(0.546, 1.638]    min      -1.285595
                  max       2.455663
                  count    26.000000
                  mean     -0.025809
Name: data2, dtype: float64

,min,max,count,mean
data1,,,,
"(-2.736, -1.639]",0.048876,0.254824,2.0,0.151850
"(-1.639, -0.547]",-1.617784,2.612791,23.0,-0.090475
"(-0.547, 0.546]",-1.684224,1.574596,49.0,-0.032136
"(0.546, 1.638]",-1.285595,2.455663,26.0,-0.025809
